# Trim 3D TIFF File with Complete Tag Preservation
This notebook provides functionality to trim a 3-dimensional TIFF file while preserving all metadata, headers, and TIFF tags, with careful handling of extratags format, and preserving the software tag, which scanimage uses for saving a number of parameters.

In [1]:
# Import required libraries
import sys
import tifffile
import numpy as np
from pathlib import Path
from io import StringIO

In [2]:
def trim_3d_tiff(input_path, output_path, start_frame, end_frame):
	"""
	Trim a 3D TIFF file while preserving all metadata and TIFF tags.

	Parameters:
	-----------
	input_path : str
		Path to the input 3D TIFF file
	output_path : str
		Path where the trimmed TIFF file will be saved
	start_frame : int
		Starting frame index (inclusive)
	end_frame : int
		Ending frame index (exclusive)

	Returns:
	--------
	None

	Raises:
	-------
	ValueError
		If the input file is not a 3D TIFF file
	FileNotFoundError
		If the input file doesn't exist
	"""
	# Convert paths to Path objects
	input_path = Path(input_path)
	output_path = Path(output_path)

	if not input_path.exists():
		raise FileNotFoundError(f"Input file not found: {input_path}")
	imagej_metadata = None
	# Read the source TIFF file
	orig_err = sys.stderr
	sys.stderr = StringIO()

	with tifffile.TiffFile(str(input_path)) as tiff:
		# Get the full data array
		data = tiff.asarray()

		# Check dimensionality
		if len(data.shape) != 3:
			raise ValueError(f"Expected 3D TIFF file, but got {len(data.shape)}D")

		# Validate frame range
		if start_frame < 0 or end_frame > data.shape[0]:
			raise ValueError(f"Invalid frame range. File has {data.shape[0]} frames.")
		if start_frame >= end_frame:
			raise ValueError("start_frame must be less than end_frame")


		# Store original page information
		original_pages = []
		for page in tiff.pages[start_frame:end_frame]:
			# Carefully extract all tags
			extratags = []
			for tag in page.tags.values():
				# Skip tags that tifffile handles automatically
				if tag.code not in [256, 257, 258, 259, 262, 277, 278, 279, 284, 317, 320, 339]:
					try:
						# Handle different tag value types
						if isinstance(tag.value, (tuple, list)):
							value = list(tag.value)  # Convert tuple to list if needed
						else:
							value = tag.value

						extratags.append((
							int(tag.code),  # Tag code/number
							tag.dtype,      # Tag dtype
							tag.count,      # Count of values
							value,          # The actual value
							False           # Save tag separately from image data
						))
					except Exception as e:
						print(f"Warning: Skipping tag {tag.code} due to error: {e}")

			original_pages.append({
				'extratags': extratags,
				'description': page.description,
				'datetime': page.datetime,
				'resolution': page.resolution,
				'compression': page.compression,
				'photometric': page.photometric,
				'planarconfig': page.planarconfig,
				'software': page.software
			})

		# Trim the data
		trimmed_data = data[start_frame:end_frame]

		# Write the trimmed file
		with tifffile.TiffWriter(str(output_path), bigtiff=tiff.is_bigtiff) as tw:
			for idx, (frame, page_info) in enumerate(zip(trimmed_data, original_pages)):
				# Write each frame with its original metadata and tags
				# print(idx)
				tw.write(
					frame,
					description=page_info['description'],
					datetime=page_info['datetime'],
					resolution=page_info['resolution'],
					compression=page_info['compression'],
					photometric=page_info['photometric'],
					planarconfig=page_info['planarconfig'],
					extratags=page_info['extratags'],
					metadata=imagej_metadata if idx == 0 else None, # ImageJ metadata only in first frame
					software=page_info['software']
				)
	sys.stderr = orig_err
	return imagej_metadata, extratags

In [3]:
# Example usage
if __name__ == "__main__":
    input_file = "/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001.tif"
    output_file = "/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001_0_6000.tif"

    # Trim to keep frames 0 through 100
    imagej_metadata, extratags = trim_3d_tiff(input_file, output_file, 0, 6000)

    # Verify the results
    verify = False

    if(verify):
        with tifffile.TiffFile(output_file) as tiff:
            print(f"Output shape: {tiff.asarray().shape}")
            if tiff.is_imagej:
                print(f"ImageJ metadata preserved: {tiff.imagej_metadata}")
            print("\nTIFF tags from first frame:")
            for tag in tiff.pages[0].tags.values():
                print(f"{tag.name}: {tag.value}")



In [5]:
imagej_metadata

In [4]:
input_path = Path(input_file)
output_path = Path(output_file)


# Read the source TIFF file
tiff = tifffile.TiffFile(str(output_path))
# Get the full data array
data = tiff.asarray()

In [5]:
start_frame = 0
end_frame = 1000

page = tiff.pages[0]
for tag in page.tags.values():
	# Skip tags that tifffile handles automatically
	if tag.code not in [256, 257, 258, 259, 262, 277, 278, 279, 284, 317, 320, 339]:
		try:
			# Handle different tag value types
			if isinstance(tag.value, (tuple, list)):
				value = list(tag.value)  # Convert tuple to list if needed
			else:
				value = tag.value
			print(tag.code)
			print(tag.name, ':')
			print(tag.value)
		except Exception as e:
			print(e)

270
ImageDescription :
frameNumbers = 1
acquisitionNumbers = 1
frameNumberAcquisition = 1
frameTimestamps_sec = 0.000000000
acqTriggerTimestamps_sec = -0.000123230
nextFileMarkerTimestamps_sec = -1.000000000
endOfAcquisition = 0
endOfAcquisitionMode = 0
dcOverVoltage = 0
epoch = [2025 11 18 15 26 16.457]
auxTrigger0 = []
auxTrigger1 = []
auxTrigger2 = []
auxTrigger3 = []
I2CData = {}
273
StripOffsets :
(39968,)
274
Orientation :
ORIENTATION.TOPLEFT
282
XResolution :
(4294967295, 361668)
283
YResolution :
(4294967295, 361668)
296
ResolutionUnit :
RESUNIT.INCH
305
Software :
SI.LINE_FORMAT_VERSION = 1
SI.PREMIUM = true
SI.TIFF_FORMAT_VERSION = 4
SI.VERSION_COMMIT = 'b0e89ff7717e60826b56173afe65ddb1ea6efb87'
SI.VERSION_MAJOR = 2023
SI.VERSION_MINOR = 1
SI.VERSION_UPDATE = 0
SI.acqState = 'grab'
SI.acqsPerLoop = 1
SI.errorMsg = ''
SI.extTrigEnable = false
SI.fieldCurvatureRxs = []
SI.fieldCurvatureRys = []
SI.fieldCurvatureTilt = 0
SI.fieldCurvatureTip = 0
SI.fieldCurvatureZs = []
SI.hBeam

In [13]:
page.software

"SI.LINE_FORMAT_VERSION = 1\nSI.PREMIUM = true\nSI.TIFF_FORMAT_VERSION = 4\nSI.VERSION_COMMIT = 'b0e89ff7717e60826b56173afe65ddb1ea6efb87'\nSI.VERSION_MAJOR = 2023\nSI.VERSION_MINOR = 1\nSI.VERSION_UPDATE = 0\nSI.acqState = 'grab'\nSI.acqsPerLoop = 1\nSI.errorMsg = ''\nSI.extTrigEnable = false\nSI.fieldCurvatureRxs = []\nSI.fieldCurvatureRys = []\nSI.fieldCurvatureTilt = 0\nSI.fieldCurvatureTip = 0\nSI.fieldCurvatureZs = []\nSI.hBeams.enablePowerBox = false\nSI.hBeams.errorMsg = ''\nSI.hBeams.flybackBlanking = true\nSI.hBeams.interlaceDecimation = [1 1]\nSI.hBeams.interlaceOffset = [0 0]\nSI.hBeams.lengthConstants = [Inf Inf]\nSI.hBeams.name = 'SI Beams'\nSI.hBeams.powerBoxEndFrame = Inf\nSI.hBeams.powerBoxStartFrame = 1\nSI.hBeams.powerBoxes = []\nSI.hBeams.powerFractionLimits = [1 1]\nSI.hBeams.powerFractions = [0.7 0]\nSI.hBeams.powers = [70 0]\nSI.hBeams.pzAdjust = [scanimage.types.BeamAdjustTypes.None scanimage.types.BeamAdjustTypes.None]\nSI.hBeams.pzFunction = {'@scanimage.util.

In [4]:
# trying to read the generated file
input_path = Path(input_file)
# output_path = Path(output_file)

if not input_path.exists():
	raise FileNotFoundError(f"Input file not found: {input_path}")

# Read the source TIFF file
tiff = tifffile.TiffFile(str(input_path))
# Get the full data array
data = tiff.asarray()


In [41]:
tiff.is_imagej

False

In [42]:
# Store original page information
original_pages = []
for page in tiff.pages[start_frame:end_frame]:
	# Carefully extract all tags
	extratags = []
	for tag in page.tags.values():
		# Skip tags that tifffile handles automatically
		if tag.code not in [256, 257, 258, 259, 262, 277, 278, 279, 284, 317, 320, 339]:
			try:
				# Handle different tag value types
				if isinstance(tag.value, (tuple, list)):
					value = list(tag.value)  # Convert tuple to list if needed
				else:
					value = tag.value

				extratags.append((
					int(tag.code),  # Tag code/number
					tag.dtype,      # Tag dtype
					tag.count,      # Count of values
					value,          # The actual value
					False           # Save tag separately from image data
				))
			except Exception as e:
				print(f"Warning: Skipping tag {tag.code} due to error: {e}")

	original_pages.append({
		'extratags': extratags,
		'description': page.description,
		'datetime': page.datetime,
		'resolution': page.resolution,
		'compression': page.compression,
		'photometric': page.photometric,
		'planarconfig': page.planarconfig
	})

In [43]:
len(extratags)

8